<a href="https://colab.research.google.com/github/mwelland/Wyrm/blob/main/demos/Allen_Cahn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Demonstration of Allen Cahn model

In [16]:
#Install Firedrake if not installed
try:
    from firedrake import *
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/firedrake-install-release-real.sh" -O "/tmp/firedrake-install.sh" && bash "/tmp/firedrake-install.sh"
    from firedrake import *

In [17]:
# Parameters
interface_width = .1
interface_energy = 1
interface_mobility = 1

In [21]:
# Create a mesh with resolution of approximately the interface width
mesh_resolution = int(1/interface_width)
mesh = UnitSquareMesh(mesh_resolution, mesh_resolution)

x = SpatialCoordinate(mesh)

In [19]:
# Define the function space and instanciate the function and test_functions
V = FunctionSpace(mesh, "CG", 1, name="phase")

phase = Function(V)
test_phase = TestFunction(V)

In [20]:
# Phase field functions
p_phase = phase**3*(6*phase**2-15*phase+10)
g_phase = phase**2*(1-phase)**2
interface_area = 3*( interface_width**2*inner(grad(phase),grad(phase)) + g_phase)

driving_force = derivative(interface_energy*interface_area, phase, test_phase)
F_phase_interface = -interface_mobility*driving_force*dx

In [26]:
# Set initial condition to be a circle in the middle of the mesh
centre = as_vector([.5,.5])
r0 = .3

def tanh_solution(x):
  return 0.5*(tanh(x/(2*interface_width)) + 1)

phase.assign(tanh_solution(norm(x-centre) - r0))

Coefficient(WithGeometry(FunctionSpace(<firedrake.mesh.MeshTopology object at 0x7aff96cdef50>, FiniteElement('Lagrange', triangle, 1), name='phase'), Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 7)), 9)